# Demo of function *display_da* from *./swiss_utils/data_cube_utilities/sdc_advutils.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy.__

Display a colored xarray.DataArray on a map and allow the user to select a given coordinates to be used later. By creating a 64 bits encoded png (the only identified way to convert a numpy array into a png without blurring it (left picture)), keeping data at pixel level (right image).

![](figures/pixelate_effect.png)

The function takes two arguments:
- **da**: xarray.DataArray to to be displayed
- **cm**: matplotlib colormap

Two objects will be created:
- **m**: map to interact with
- **dc**: draw control  to point a location (using the dedicated circlemarker button on the left of the map) to be used later on

__Warning !__
By default web browser will add blur to displayed images faster. Then we need to apply a dedicated css parameter to the present jupyter notebook by running a cell located at the beginning of the notebook and containing the following code:
*****
```html
# css tweek to avoid browser blurry png (need to be run before function cell)
# source: https://github.com/python-visualization/folium/blob/master/folium/raster_layers.py
from IPython.core.display import HTML
HTML("""
<style>
    .leaflet-image-layer {
        /* old android/safari*/
        image-rendering: -webkit-optimize-contrast;
        image-rendering: crisp-edges; /* safari */
        image-rendering: pixelated; /* chrome */
        image-rendering: -moz-crisp-edges; /* firefox */
        image-rendering: -o-crisp-edges; /* opera */
        -ms-interpolation-mode: nearest-neighbor; /* ie */
    }
</style>
""")
```
*****

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `display_da?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_advutils.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import numpy as np
import xarray as xr

from datetime import datetime

import datacube
dc = datacube.Datacube()

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_advutils import display_da

In [ ]:
# Cell mentionned in the documentation above

# css tweek to avoid browser blurry png (need to be run before function cell)
# source: https://github.com/python-visualization/folium/blob/master/folium/raster_layers.py
from IPython.core.display import HTML
HTML("""
<style>
    .leaflet-image-layer {
        /* old android/safari*/
        image-rendering: -webkit-optimize-contrast;
        image-rendering: crisp-edges; /* safari */
        image-rendering: pixelated; /* chrome */
        image-rendering: -moz-crisp-edges; /* firefox */
        image-rendering: -o-crisp-edges; /* opera */
        -ms-interpolation-mode: nearest-neighbor; /* ie */
    }
</style>
""")

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands __(as this notebook will compute NDVI you will need nir, red and the mask)__

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

In [ ]:
%load config_cell.txt

In [ ]:
draw_map([min_lat, max_lat], [min_lon, max_lon], draw = False)[0]

In [ ]:
# Create an average NDVI to display

dataset_clean, clean_mask = load_multi_clean(dc = dc, products = product, time = [start_date, end_date],
                                          lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                          measurements = measurements)

ndvi = (dataset_clean.nir - dataset_clean.red) / (dataset_clean.nir + dataset_clean.red)
# del dataset_clean
del clean_mask
ndvi_mean = np.nanmean(ndvi.values, axis=0)
ndvi_mean = xr.DataArray(ndvi_mean, dims=['latitude', 'longitude']).astype(np.float64)
ndvi_mean = ndvi_mean.assign_coords(latitude=ndvi.latitude, longitude=ndvi.longitude)

# replace +-Inf by nan
ndvi_mean = ndvi_mean.where(np.isfinite(ndvi_mean))

# keep the range of values within +-1
ndvi_mean = ndvi_mean.where(ndvi_mean.values > -1, -1). \
                 where(ndvi_mean.values < 1, 1)

In [ ]:
ndvi_mean = ndvi_mean.where(ndvi_mean.values > -1, -1). \
                 where(ndvi_mean.values < 1, 1)

In [ ]:
# Let's first use a default matplotlib colormap ('_r' invert the colormap)
# catalog at https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

from matplotlib import cm

m1, pos1, io1 = display_da(ndvi_mean,
                        cm = cm.get_cmap('ocean_r', 256))
m1

In [ ]:
# As no default colormap fits NDVI display, let's create one manually

from matplotlib import colors

m2, pos2, io2 = display_da(ndvi_mean,
                    cm = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue','lightgreen','darkgreen'], N=256))
m2

In [ ]:
# Just for fun, link the two map (pan and zoom will apply on both maps)

from traitlets import link

map_center_link = link((m1, 'center'), (m2, 'center'))
map_zoom_link = link((m1, 'zoom'), (m2, 'zoom'))

In [ ]:
# You can also combine the two maps (they will still be linked as well with the two previous map)

import ipywidgets
 
ipywidgets.HBox([m1, m2])

In [ ]:
# You can also control the first map opacity from this cell (for fun we will also link maps opacity)

map_opacity_link = link((io1, 'opacity'), (io2, 'opacity'))
io1.interact(opacity=(0.0,1.0,0.01))

In [ ]:
# Use the circlemarker button of the second map (right) to select a location and
# run this cell to get its coordinates
location = pos2.last_draw['geometry']['coordinates']
print(location)